In [9]:
import numpy as np, math, pandas as pd
from scipy.stats import multivariate_normal
from decimal import Decimal

def clustering(points):
    points = list(set(points)) #To eliminate reduntant points

    '''
    Initialize random gaussians!
    '''
    mean = [np.array([5,4,1.5,0.4]), np.array([4.5,5,0.5,0.5]), np.array([6,3,2,1])]
    covariance1 = np.array([[16,0,0,0], [0,4,0,0], [0,0,9,0], [0,0,0,1]])
    covariance2 = np.array([[4,0,0,0], [0,16,0,0], [0,0,1,0], [0,0,0,9]])
    covariance3 = np.array([[16,0,0,0], [0,1,0,0], [0,0,9,0], [0,0,0,4]])
    covariance = [covariance1, covariance2, covariance3]
    weights = np.array([0.3, 0.3, 0.4])

    init_log_likelihood = None #Log likelihood to test convergence None initially
    threshold = 0.000000000001 #Threshold difference for convergence

    iterations = 0 #for counting iterations

    while True:
        iterations +=1 

        prob_dict = {point:[] for point in points}
        new_means = [np.array([0,0,0,0]) for _ in range(3)]
        new_covariance = [np.array([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]) for _ in range(3)]

        log_likelihood = Decimal(0) #Using decimal to increase the precision of floating point numbers

        for point in points:

            prob_sum = 0 #Prob sum for normalization

            for _ in range(3):
                prob =  multivariate_normal.pdf(point, mean[_], covariance[_], allow_singular=True) #Finding gaussian probability
                prob = np.dot(weights[_], prob)+1 #Multiply gaussian probability with weight
                prob_dict[point].append(prob)
                prob_sum += prob
            
            prob_sum = Decimal(prob_sum)

            log_likelihood += prob_sum.ln() #comprehensive computation of log likelihood

            for _ in range(3):
                prob_dict[point][_] = prob_dict[point][_]/float(prob_sum) #Normalizing the probability
                new_means[_] = new_means[_]+np.dot(prob_dict[point][_], list(point)) #New means for the next iteration

        for _ in range(3):
            weights[_] = sum([prob_item[1][_] for prob_item in prob_dict.items()]) #New weights
            new_means[_] = np.divide(new_means[_], weights[_])
            #New covariance
            new_covariance[_] = new_covariance[_]+np.dot(np.dot(prob_dict[point][_], (np.array(point)-new_means[_])),  np.transpose(np.array(point)-new_means[_]))
        
        for _ in range(3):
            new_covariance[_] = np.divide(new_covariance[_],weights[_])
            weights[_] = weights[_]/len(points)        
        
        mean = new_means 
        covariance = new_covariance

        #check for convergence
        if init_log_likelihood and abs(init_log_likelihood-log_likelihood)<=threshold:
            break

        init_log_likelihood = log_likelihood

    print("Means: ", mean)
    print("Covariance: ", covariance)
    print("Log-likehilhood: ", log_likelihood)
    print("Iterations: ", iterations)

def get_points():
    '''
    Read points from csv file.
    '''
    file = pd.read_csv("dataset.csv")
    points = []
    for _,row in file.iterrows():
        points.append((row['A'], row['B'], row['C'], row['D']))

    return points
    
def main():
    points = get_points()
    clustering(points)

if __name__ == '__main__':
    main()


Means:  [array([5.85646258, 3.05578231, 3.78027211, 1.20884354]), array([5.85646259, 3.05578231, 3.78027211, 1.20884354]), array([5.85646259, 3.05578231, 3.78027211, 1.20884354])]
Covariance:  [array([[0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078]]), array([[0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078]]), array([[0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078],
       [0.00489078, 0.00489078, 0.00489078, 0.00489078]])]
Log-likehilhood:  168.6804828730073817347056082
Iterations:  6
